# Neural Net Creation

In this notebook, I created my neural model to train on my image datasets. 

In [ ]:
import pandas as pd
from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Model

%pylab inline

## Loading Training and Validation Images

Created ImageDataGenerator objects to pull in my training and validation datasets from my directory.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range = 0.1, horizontal_flip = True, vertical_flip = True)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255, zoom_range = 0.1, horizontal_flip = True, vertical_flip = True)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'Input your training directory pathname here',
    target_size = (224, 224),
    batch_size = 128,
    class_mode = 'binary',
)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    'Input your training directory pathname here',
    target_size = (224, 224),
    batch_size = 128,
    class_mode = 'binary',
    
)

## Neural Net Creation

Created a sequential neural net with 4 convolutions. 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (224, 224, 3)))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.85))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

## Training the Neural Net Model

Used ModelCheckpoint and EarlyStopping callbacks to obtain the best model weights/epochs.

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [ ]:
earlystopping_callback = EarlyStopping(
    monitor = 'val_accuracy',
    patience = 4,
    verbose = 1,
)

In [ ]:
checkpoint_filepath = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = True,
    save_best_only = True,
    monitor = 'val_accuracy',
    verbose = 1
)

In [ ]:
history = model.fit(
    train_generator,
    epochs = 10,
    validation_data = val_generator,
    verbose = True,
    validation_steps = 2478//128,
    steps_per_epoch = 10941//128,
    class_weight = {1:4., 0:1.},
    callbacks = [checkpoint_callback,earlystopping_callback]
)

In [ ]:
plt.plot(history.history['accuracy'], 'ko')
plt.plot(history.history['val_accuracy'], 'b')

plt.title('Accuracy vs Training Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation']);

Save the model to load in with the best weight.

In [ ]:
model.save('model.h5')

## Model Results with Testing Dataset

Load the weights in the model and evaluate on the testing dataset.

In [ ]:
model.load_weights('weights-improvement-08-0.98.hdf5')

In [ ]:
model.summary()

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1. / 255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    '/home/rw1219/Test',
    target_size = (224, 224),
    batch_size = 128,
    class_mode = 'binary',
    shuffle = False
)

In [ ]:
scores = model.evaluate(test_generator)

## Classification Report and Confusion Matrix

Generated a classification report to see the precision, recall, and f1 scores.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict_classes(test_generator)

In [ ]:
print(classification_report(y_pred, test_generator.classes))

Looked at a confusion matrix to see if the model predicted well.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_pred, test_generator.classes)